<a href="https://colab.research.google.com/github/Arminsbss/Simplicial-Neural-Network/blob/main/impute_citations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional
import torch.utils.data as data
import numpy as np
import sys

!git clone https://github.com/stefaniaebli/simplicial_neural_networks.git


Cloning into 'simplicial_neural_networks'...
remote: Enumerating objects: 339, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 339 (delta 50), reused 48 (delta 48), pack-reused 280
Receiving objects: 100% (339/339), 2.42 MiB | 16.10 MiB/s, done.
Resolving deltas: 100% (189/189), done.


In [ ]:
# %cd /content/simplicial_neural_networks/scnn
# from simplicial_neural_networks import scnn
import simplicial_neural_networks.scnn
import simplicial_neural_networks.scnn.chebyshev


In [ ]:
#@title scnn
#scnn
import torch
import torch.nn as nn
import numpy as np
import scipy.sparse as sp

import simplicial_neural_networks.scnn.chebyshev
def coo2tensor(A):
    assert(sp.isspmatrix_coo(A))
    idxs = torch.LongTensor(np.vstack((A.row, A.col)))
    vals = torch.FloatTensor(A.data)
    return torch.sparse_coo_tensor(idxs, vals, size = A.shape, requires_grad = False)

class SimplicialConvolution(nn.Module):
    def __init__(self, K, C_in, C_out, enable_bias = True, variance = 1.0, groups = 1):
        assert groups == 1, "Only groups = 1 is currently supported."
        super().__init__()

        assert(C_in > 0)
        assert(C_out > 0)
        assert(K > 0)

        self.C_in = C_in
        self.C_out = C_out
        self.K = K
        self.enable_bias = enable_bias

        self.theta = nn.parameter.Parameter(variance*torch.randn((self.C_out, self.C_in, self.K)))
        if self.enable_bias:
            self.bias = nn.parameter.Parameter(torch.zeros((1, self.C_out, 1)))
        else:
            self.bias = 0.0

    def forward(self, L, x):
        assert(len(L.shape) == 2)
        assert(L.shape[0] == L.shape[1])

        (B, C_in, M) = x.shape

        assert(M == L.shape[0])
        assert(C_in == self.C_in)

        X = scnn.chebyshev.assemble(self.K, L, x)
        y = torch.einsum("bimk,oik->bom", (X, self.theta))
        assert(y.shape == (B, self.C_out, M))

        return y + self.bias

# This class does not yet implement the
# Laplacian-power-pre/post-composed with the coboundary. It can be
# simulated by just adding more layers anyway, so keeping it simple
# for now.
#
# Note: You can use this for a adjoints of coboundaries too. Just feed
# a transposed D.
class Coboundary(nn.Module):
    def __init__(self, C_in, C_out, enable_bias = True, variance = 1.0):
        super().__init__()

        assert(C_in > 0)
        assert(C_out > 0)

        self.C_in = C_in
        self.C_out = C_out
        self.enable_bias = enable_bias

        self.theta = nn.parameter.Parameter(variance*torch.randn((self.C_out, self.C_in)))
        if self.enable_bias:
            self.bias = nn.parameter.Parameter(torch.zeros((1, self.C_out, 1)))
        else:
            self.bias = 0.0

    def forward(self, D, x):
        assert(len(D.shape) == 2)

        (B, C_in, M) = x.shape

        assert(D.shape[1] == M)
        assert(C_in == self.C_in)

        N = D.shape[0]

        # This is essentially the equivalent of chebyshev.assemble for
        # the convolutional modules.
        X = []
        for b in range(0, B):
            X12 = []
            for c_in in range(0, self.C_in):
                X12.append(D.mm(x[b, c_in, :].unsqueeze(1)).transpose(0,1)) # D.mm(x[b, c_in, :]) has shape Nx1
            X12 = torch.cat(X12, 0)

            assert(X12.shape == (self.C_in, N))
            X.append(X12.unsqueeze(0))

        X = torch.cat(X, 0)
        assert(X.shape == (B, self.C_in, N))

        y = torch.einsum("oi,bin->bon", (self.theta, X))
        assert(y.shape == (B, self.C_out, N))

        return y + self.bias

In [ ]:
#@title chebyshef
#chebyshef
import torch
import scipy.sparse as sp
import scipy.sparse.linalg as spl
import numpy as np
def normalize(L, half_interval = False):
    assert(sp.isspmatrix(L))
    M = L.shape[0]
    assert(M == L.shape[1])
    topeig = spl.eigsh(L, k=1, which="LM", return_eigenvectors = False)[0]
    #print("Topeig = %f" %(topeig))

    ret = L.copy()
    if half_interval:
        ret *= 1.0/topeig
    else:
        ret *= 2.0/topeig
        ret.setdiag(ret.diagonal(0) - np.ones(M), 0)

    return ret

def assemble(K, L, x):
    (B, C_in, M) = x.shape
    assert(L.shape[0] == M)
    assert(L.shape[0] == L.shape[1])
    assert(K > 0)

    X = []
    for b in range(0, B):
        X123 = []
        for c_in in range(0, C_in):
            X23 = []
            X23.append(x[b, c_in, :].unsqueeze(1)) # Constant, k = 0 term.

            if K > 1:
                X23.append(L.mm(X23[0]))
            for k in range(2, K):
                X23.append(2*(L.mm(X23[k-1])) - X23[k-2])

            X23 = torch.cat(X23, 1)
            assert(X23.shape == (M, K))
            X123.append(X23.unsqueeze(0))

        X123 = torch.cat(X123, 0)
        assert(X123.shape == (C_in, M, K))
        X.append(X123.unsqueeze(0))

    X = torch.cat(X, 0)
    assert(X.shape == (B, C_in, M, K))

    return X

In [ ]:

class MySCNN(nn.Module):
    def __init__(self, colors = 1):
        super().__init__()

        assert(colors > 0)
        self.colors = colors

        num_filters = 30 #20
        variance = 0.01 #0.001

        # Degree 0 convolutions.
        self.C0_1 = scnn.scnn.SimplicialConvolution(5, self.colors, num_filters*self.colors, variance=variance)
        self.C0_2 = scnn.scnn.SimplicialConvolution(5, num_filters*self.colors, num_filters*self.colors, variance=variance)
        self.C0_3 = scnn.scnn.SimplicialConvolution(5, num_filters*self.colors, self.colors, variance=variance)

        # Degree 1 convolutions.
        self.C1_1 = scnn.scnn.SimplicialConvolution(5, self.colors, num_filters*self.colors, variance=variance)
        self.C1_2 = scnn.scnn.SimplicialConvolution(5, num_filters*self.colors, num_filters*self.colors, variance=variance)
        self.C1_3 = scnn.scnn.SimplicialConvolution(5, num_filters*self.colors, self.colors, variance=variance)

        # Degree 2 convolutions.
        self.C2_1 = scnn.scnn.SimplicialConvolution(5, self.colors, num_filters*self.colors, variance=variance)
        self.C2_2 = scnn.scnn.SimplicialConvolution(5, num_filters*self.colors, num_filters*self.colors, variance=variance)
        self.C2_3 = scnn.scnn.SimplicialConvolution(5, num_filters*self.colors, self.colors, variance=variance)



    def forward(self, Ls, Ds, adDs, xs):
        assert(len(xs) == 3) # The three degrees are fed together as a list.

        assert(len(Ls) == len(Ds))
        Ms = [L.shape[0] for L in Ls]
        Ns = [D.shape[0] for D in Ds]

        Bs = [x.shape[0] for x in xs]
        C_ins = [x.shape[1] for x in xs]
        Ms = [x.shape[2] for x in xs]

        assert(Ms == [D.shape[1] for D in Ds])
        assert(Ms == [L.shape[1] for L in Ls])
        assert([adD.shape[0] for adD in adDs] == [D.shape[1] for D in Ds])
        assert([adD.shape[1] for adD in adDs] == [D.shape[0] for D in Ds])

        assert(Bs == len(Bs)*[Bs[0]])
        assert(C_ins == len(C_ins)*[C_ins[0]])

        out0_1 = self.C0_1(Ls[0], xs[0]) #+ self.D10_1(xs[1])
        out1_1 = self.C1_1(Ls[1], xs[1]) #+ self.D01_1(xs[0]) + self.D21_1(xs[2])
        out2_1 = self.C2_1(Ls[2], xs[2]) #+ self.D12_1(xs[1])

        out0_2 = self.C0_2(Ls[0], nn.LeakyReLU()(out0_1)) #+ self.D10_2(nn.LeakyReLU()(out1_1))
        out1_2 = self.C1_2(Ls[1], nn.LeakyReLU()(out1_1)) #+ self.D01_2(nn.LeakyReLU()(out0_1)) + self.D21_2(nn.LeakyReLU()(out2_1))
        out2_2 = self.C2_2(Ls[2], nn.LeakyReLU()(out2_1)) #+ self.D12_2(nn.LeakyReLU()(out1_1))

        out0_3 = self.C0_3(Ls[0], nn.LeakyReLU()(out0_2)) #+ self.D10_3(nn.LeakyReLU()(out1_2))
        out1_3 = self.C1_3(Ls[1], nn.LeakyReLU()(out1_2)) #+ self.D01_3(nn.LeakyReLU()(out0_2)) + self.D21_2(nn.LeakyReLU()(out2_2))
        out2_3 = self.C2_3(Ls[2], nn.LeakyReLU()(out2_2)) #+ self.D12_3(nn.LeakyReLU()(out1_2))

        #return [out0_3, torch.zeros_like(xs[1]), torch.zeros_like(xs[2])]
        #return [torch.zeros_like(xs[0]), out1_3, torch.zeros_like(xs[2])]
        return [out0_3, out1_3, out2_3]

In [ ]:
import sys
sys.path.append('.')

In [ ]:
%cd /content/simplicial_neural_networks/data

/content/simplicial_neural_networks/data


In [ ]:
logdir

NameError: ignored

In [ ]:
%cd /content/simplicial_neural_networks/data/s2_3_collaboration_complex
laplacians = np.load('150250_laplacians.npy',allow_pickle=True)

/content/simplicial_neural_networks/data/s2_3_collaboration_complex


In [ ]:
print(laplacians)

[<352x352 sparse matrix of type '<class 'numpy.float32'>'
 	with 3300 stored elements in COOrdinate format>
 <1474x1474 sparse matrix of type '<class 'numpy.float32'>'
 	with 25466 stored elements in COOrdinate format>
 <3285x3285 sparse matrix of type '<class 'numpy.float32'>'
 	with 4697 stored elements in COOrdinate format>
 <5019x5019 sparse matrix of type '<class 'numpy.float32'>'
 	with 5419 stored elements in COOrdinate format>
 <5559x5559 sparse matrix of type '<class 'numpy.float32'>'
 	with 5601 stored elements in COOrdinate format>
 <4547x4547 sparse matrix of type '<class 'numpy.float32'>'
 	with 4547 stored elements in COOrdinate format>
 <2732x2732 sparse matrix of type '<class 'numpy.float32'>'
 	with 2732 stored elements in COOrdinate format>
 <1175x1175 sparse matrix of type '<class 'numpy.float32'>'
 	with 1175 stored elements in COOrdinate format>
 <343x343 sparse matrix of type '<class 'numpy.float32'>'
 	with 343 stored elements in COOrdinate format>
 <61x61 sparse

In [ ]:
print(laplacians[1])

  (0, 0)	6.0
  (0, 63)	-1.0
  (0, 64)	-1.0
  (0, 65)	-1.0
  (0, 66)	-1.0
  (0, 67)	-1.0
  (0, 68)	-1.0
  (0, 69)	-1.0
  (0, 70)	-1.0
  (0, 71)	-1.0
  (0, 72)	-1.0
  (0, 73)	-1.0
  (0, 74)	-1.0
  (0, 75)	-1.0
  (0, 76)	-1.0
  (0, 77)	-1.0
  (0, 78)	-1.0
  (0, 79)	-1.0
  (0, 80)	-1.0
  (0, 81)	-1.0
  (0, 82)	-1.0
  (0, 83)	-1.0
  (0, 84)	-1.0
  (0, 85)	-1.0
  (0, 86)	-1.0
  :	:
  (1471, 1471)	11.0
  (1471, 139)	1.0
  (1471, 470)	1.0
  (1471, 555)	1.0
  (1471, 804)	1.0
  (1471, 1201)	1.0
  (1471, 1357)	1.0
  (1471, 1373)	1.0
  (1471, 1408)	1.0
  (1471, 1473)	-1.0
  (1472, 1472)	7.0
  (1473, 29)	-1.0
  (1473, 154)	-1.0
  (1473, 470)	-1.0
  (1473, 555)	-1.0
  (1473, 771)	-1.0
  (1473, 804)	-1.0
  (1473, 1087)	-1.0
  (1473, 1213)	-1.0
  (1473, 1294)	-1.0
  (1473, 1320)	-1.0
  (1473, 1373)	-1.0
  (1473, 1408)	-1.0
  (1473, 1471)	-1.0
  (1473, 1473)	7.0


In [ ]:
# the last one
print(laplacians[1].getrow(1471).getcol(1408).toarray()[0][0])

4.0


In [ ]:
print(type(laplacians[1]))

<class 'scipy.sparse._coo.coo_matrix'>


In [ ]:
laplacians[1].data[(laplacians[1].row==1471) & (laplacians[1].col==1408)]=4.0


In [ ]:
from scipy.sparse import csr_matrix
# Create the sparse matrix
data = [-1.0, -1.0, -1.0, 5.0, -1.0, -1.0]
indices = [63, 64, 65, 66, 67, 68]
indptr = [0, len(indices)]
sparse_matrix = csr_matrix((data, indices, indptr), shape=(1, 69))

# Replace the element at coordinate (0,66) with 7.0
sparse_matrix[0, 66] = 7.0

# Print the whole sparse matrix
for i in range(sparse_matrix.shape[1]):
    if sparse_matrix[0,i] != 0:
        print(f"({sparse_matrix.indices[i]}, {i}) {sparse_matrix[0,i]}")

IndexError: ignored

In [ ]:
print(type(sparse_matrix))

<class 'scipy.sparse._csr.csr_matrix'>


In [ ]:
print(type(laplacians[1]))

<class 'scipy.sparse._coo.coo_matrix'>


In [ ]:
laplacians[1].data[0]

6.0

In [ ]:
laplacians[1].shape

(1474, 1474)

In [ ]:
# from torch._C import float32
for i in range(25466):
  laplacians[1].data[i] /= 2


In [ ]:
print(laplacians[1])

  (0, 0)	3.0
  (0, 63)	-0.5
  (0, 64)	-0.5
  (0, 65)	-0.5
  (0, 66)	-0.5
  (0, 67)	-0.5
  (0, 68)	-0.5
  (0, 69)	-0.5
  (0, 70)	-0.5
  (0, 71)	-0.5
  (0, 72)	-0.5
  (0, 73)	-0.5
  (0, 74)	-0.5
  (0, 75)	-0.5
  (0, 76)	-0.5
  (0, 77)	-0.5
  (0, 78)	-0.5
  (0, 79)	-0.5
  (0, 80)	-0.5
  (0, 81)	-0.5
  (0, 82)	-0.5
  (0, 83)	-0.5
  (0, 84)	-0.5
  (0, 85)	-0.5
  (0, 86)	-0.5
  :	:
  (1471, 1471)	5.5
  (1471, 139)	0.5
  (1471, 470)	0.5
  (1471, 555)	0.5
  (1471, 804)	0.5
  (1471, 1201)	0.5
  (1471, 1357)	0.5
  (1471, 1373)	0.5
  (1471, 1408)	0.5
  (1471, 1473)	-0.5
  (1472, 1472)	3.5
  (1473, 29)	-0.5
  (1473, 154)	-0.5
  (1473, 470)	-0.5
  (1473, 555)	-0.5
  (1473, 771)	-0.5
  (1473, 804)	-0.5
  (1473, 1087)	-0.5
  (1473, 1213)	-0.5
  (1473, 1294)	-0.5
  (1473, 1320)	-0.5
  (1473, 1373)	-0.5
  (1473, 1408)	-0.5
  (1473, 1471)	-0.5
  (1473, 1473)	3.5


In [ ]:
%cd /content/simplicial_neural_networks
np.save('150250_laplacians.npy', laplacians)


/content/simplicial_neural_networks


In [ ]:
def main():
    torch.manual_seed(1337)
    np.random.seed(1337)


    prefix = sys.argv[1] ##input

    logdir = sys.argv[2] ##output
    starting_node=150250
    percentage_missing_values=30
    cuda = False

    topdim = 2
    %cd '/content/simplicial_neural_networks/data/s2_3_collaboration_complex'

    laplacians = np.load('{}_laplacians.npy'.format(starting_node),allow_pickle=True)
    boundaries = np.load('{}_boundaries.npy'.format(starting_node),allow_pickle=True)
    %cd /content/simplicial_neural_networks


    Ls =[coo2tensor(chebyshev.normalize(laplacians[i],half_interval=True)) for i in range(topdim+1)] #####scnn.chebyshev.normalize ?
    Ds=[coo2tensor(boundaries[i].transpose()) for i in range(topdim+1)]
    adDs=[coo2tensor(boundaries[i]) for i in range(topdim+1)]


    network = MySCNN(colors = 1)


    learning_rate = 0.001
    optimizer = torch.optim.Adam(network.parameters(), lr=learning_rate)
    criterion = nn.L1Loss(reduction="sum")
    #criterion = nn.MSELoss(reduction="sum")

    batch_size = 1

    num_params = 0
    print("Parameter counts:")
    for param in network.parameters():
        p = np.array(param.shape, dtype=int).prod()
        print(p)
        num_params += p
    print("Total number of parameters: %d" %(num_params))

    %cd /content/simplicial_neural_networks/data/s2_3_collaboration_complex
    masks_all_deg = np.load('{}_percentage_{}_known_values.npy'.format(starting_node,percentage_missing_values),allow_pickle=True) ## positive mask= indices that we keep ##1 mask #entries 0 degree
    masks=[list(masks_all_deg[i].values()) for i in range(len(masks_all_deg))]
    # losslogf = open("%s/loss.txt" %(logdir), "w")

    cochain_target_alldegs = []
    signal = np.load('{}_cochains.npy'.format(starting_node),allow_pickle=True)
    raw_data=[list(signal[i].values()) for i in range(len(signal))]
    for d in range(0, topdim+1):
        cochain_target = torch.zeros((batch_size, 1, len(raw_data[d])), dtype=torch.float, requires_grad = False)
        for i in range(0, batch_size):
            cochain_target[i, 0, :] = torch.tensor(raw_data[d], dtype=torch.float, requires_grad = False)

        cochain_target_alldegs.append(cochain_target)

    cochain_input_alldegs = []
    signal = np.load('{}_percentage_{}_input_damaged.npy'.format(starting_node,percentage_missing_values),allow_pickle=True)
    raw_data=[list(signal[i].values()) for i in range(len(signal))]
    for d in range(0, topdim+1):

        cochain_input = torch.zeros((batch_size, 1, len(raw_data[d])), dtype=torch.float, requires_grad = False)

        for i in range(0, batch_size):
            cochain_input[i, 0, :] = torch.tensor(raw_data[d], dtype=torch.float, requires_grad = False)

        cochain_input_alldegs.append(cochain_input)

    #cochain_target_alldegs[0] = torch.zeros_like(cochain_target_alldegs[0])
    #cochain_target_alldegs[2] = torch.zeros_like(cochain_target_alldegs[2])

    #cochain_input_alldegs[0] = torch.zeros_like(cochain_input_alldegs[0])
    #cochain_input_alldegs[2] = torch.zeros_like(cochain_input_alldegs[2])

    print([float(len(masks[d]))/float(len(cochain_target_alldegs[d][0,0,:])) for d in range(0,2+1)])

    for i in range(0, 1000):
        xs = [cochain_input.clone() for cochain_input in cochain_input_alldegs]

        optimizer.zero_grad()
        ys = network(Ls, Ds, adDs, xs)

        loss = torch.FloatTensor([0.0])
        for b in range(0, batch_size):
            for d in range(0, topdim+1):
                loss += criterion(ys[d][b, 0, masks[d]], cochain_target_alldegs[d][b, 0, masks[d]])

        detached_ys = [ys[d].detach() for d in range(0, topdim+1)]

        # if np.mod(i, 10) == 0:
        #     for d in range(0,topdim+1):
        #         np.savetxt("%s/output_%d_%d.txt" %(logdir, i, d), detached_ys[d][0,0,:])

        for d in range(0, topdim+1):
            predictionlogf = open("%s/prediction_%d_%d.txt" %(logdir, i, d), "w")
            actuallogf = open("%s/actual_%d_%d.txt" %(logdir, i, d), "w")

            for b in range(0, batch_size):
                for y in detached_ys[d][b, 0, masks[d]]:
                    predictionlogf.write("%f " %(y))
                predictionlogf.write("\n")
                for x in cochain_target_alldegs[d][b, 0, masks[d]]:
                    actuallogf.write("%f " %(x))
                actuallogf.write("\n")
            predictionlogf.close()
            actuallogf.close()


        losslogf.write("%d %f\n" %(i, loss.item()))
        losslogf.flush()

        loss.backward()
        optimizer.step()

    # losslogf.close()

    name_networks=['C0_1,C0_2','C0_3','C1_1,C1_2','C1_3', 'C2_1,C2_2','C2_3']



if __name__ == "__main__":
    main()

/content/simplicial_neural_networks/data/s2_3_collaboration_complex
/content/simplicial_neural_networks
Parameter counts:
150
30
4500
30
150
1
150
30
4500
30
150
1
150
30
4500
30
150
1
Total number of parameters: 14583
/content/simplicial_neural_networks/data/s2_3_collaboration_complex
[0.6988636363636364, 0.6994572591587517, 0.6998477929984779]


NameError: ignored

In [ ]:
name_networks=['C0_1,C0_2','C0_3','C1_1,C1_2','C1_3', 'C2_1,C2_2','C2_3']



if __name__ == "__main__":
    main()

/content/simplicial_neural_networks/data/s2_3_collaboration_complex
/content/simplicial_neural_networks
Parameter counts:
150
30
4500
30
150
1
150
30
4500
30
150
1
150
30
4500
30
150
1
Total number of parameters: 14583
/content/simplicial_neural_networks/data/s2_3_collaboration_complex
[0.6988636363636364, 0.6994572591587517, 0.6998477929984779]


KeyboardInterrupt: ignored